In [ ]:
import requests
def returnUrl(search_keyword):
    headers = {
      'Content-Type': "application/json"
    }
    body = {
    "query": search_keyword
    }
    response = requests.post("{url}/{page}".format(url="https://us-central1-burner-kunwadhw2.cloudfunctions.net",page="function-1"), json=body, headers=headers)
    return response.json()["link"]

#print(returnUrl("Kunal Wadhwa"))

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession \
.builder \
.appName("udf job") \
.getOrCreate()

schema = StructType([
    StructField('EMPLOYEE_ID',
                IntegerType(), True),
    StructField('FIRST_NAME',
                StringType(), True),
    StructField('LAST_NAME',
                StringType(), True),
    StructField('EMAIL',
                StringType(), True),
    StructField('PHONE_NUMBER',
                StringType(), True),
    StructField('HIRE_DATE',
                StringType(), True),
    StructField('JOB_ID',
                StringType(), True),
    StructField('SALARY',
                IntegerType(), True),
    StructField('COMMISSION_PCT',
                StringType(), True),
    StructField('MANAGER_ID',
                StringType(), True),
    StructField('DEPARTMENT_ID',
                IntegerType(), True)
])

df = spark \
.read \
.format("csv") \
.schema(schema) \
.option("header",True) \
.load("gs://kunal-bucket/data/employees.csv")

df.select("EMPLOYEE_ID","FIRST_NAME","LAST_NAME").show()

In [ ]:
from pyspark.sql.functions import col,udf,concat,lit
from pyspark.sql.types import StringType

getUrl = udf(lambda z:returnUrl(z),StringType())

df_with_url = df \
    .withColumn("FULL_NAME",concat(concat(col("FIRST_NAME"),lit(" ")),col("LAST_NAME"))) \
    .withColumn("URL",getUrl(concat(concat(col("FIRST_NAME"),lit(" ")),col("LAST_NAME"))))

df_with_url.select("EMPLOYEE_ID","FIRST_NAME","LAST_NAME","FULL_NAME","URL").show()